In [27]:
from pocketflow import Node, Flow
from langchain_core.tools import tool
from agentic_blocks.utils.tools_utils import create_tool_registry, execute_pending_tool_calls
from agentic_blocks import call_llm, Messages

In [28]:
@tool
def add(a: float, b: float) -> float:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

@tool
def divide(a: float, b: float) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

tools = [add, multiply, divide]

tool_registry = create_tool_registry(tools)

In [23]:
class LLMNode(Node):
    def prep(self, shared):
        return shared["messages"]

    def exec(self, messages) -> str:
        model = "qwen/qwen3-235b-a22b-2507"
        response = call_llm(model=model, messages=messages, tools=tools)
        messages.add_response_message(response)
        return messages
    
    def post(self, shared,  prep_res, messages):
        if messages.has_pending_tool_calls():
            return "tool_node"
        else:
            return "answer_node"   
        
class ToolNode(Node):
    
    def prep(self, shared):
        return shared["messages"]

    def exec(self, messages) -> str:
        print(f"Tool node: {messages.get_pending_tool_calls()}")
        tool_responses = execute_pending_tool_calls(messages, tool_registry)
        messages.add_tool_responses(tool_responses)
        return messages
    
    def post(self, shared,  prep_res, messages):
        return "llm_node"
    
class AnswerNode(Node):
    
    def prep(self, shared):
        messages = shared["messages"]
        shared["answer"] = messages.get_messages()[-1]['content']
        return messages

In [24]:
llm_node = LLMNode()
tool_node = ToolNode()
answer_node = AnswerNode()

llm_node - "tool_node" >> tool_node
tool_node - "llm_node" >> llm_node
llm_node - "answer_node" >> answer_node


flow = Flow(llm_node)

In [25]:
messages = Messages(
    system_prompt="You are a helpful assistant tasked with writing performing arithmetic on a set of inputs.",
    user_prompt="Add 3 and 4. Multiply the output by 2. Divide the output by 5"
)

shared = {"messages": messages}

In [26]:
flow.run(shared)

print(shared["answer"])

Tool node: [{'tool_name': 'add', 'arguments': {'a': 3, 'b': 4}, 'tool_call_id': '20f1c6176'}]
Tool node: [{'tool_name': 'multiply', 'arguments': {'a': 7, 'b': 2}, 'tool_call_id': '38d31c457'}]
Tool node: [{'tool_name': 'divide', 'arguments': {'a': 14, 'b': 5}, 'tool_call_id': '38ba4b155'}]
The final result is 2.8.


In [49]:
messages.add_user_message("add 3 to that")


In [50]:
flow.run(shared)

Tool node: [{'tool_name': 'add', 'arguments': {'a': 2.8, 'b': 3}, 'tool_call_id': 'call_93aa4159bf1d44168a64de'}]
Answer: The final result is 5.8.
